Splitting text into paragraphs

Import libraries

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=a0fadd595dcc2edcebd56c60aaeeafb7a237a3e614e1bda7e12bdc116cf31098
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
# First let import the most necessary libs
import pandas as pd
import numpy as np
# Library to import pre-trained model for sentence embeddings
from sentence_transformers import SentenceTransformer
# Calculate similarities between sentences
from sklearn.metrics.pairwise import cosine_similarity
# Visualization library
import seaborn as sns
import matplotlib.pyplot as plt
# package for finding local minimas
from scipy.signal import argrelextrema
import math

In [ ]:
df_all = pd.DataFrame()

Load text

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hurricanes.csv to hurricanes.csv


In [ ]:
import pandas as pd

df_combined = pd.read_csv('hurricanes.csv')

# df_combined = df_combined.drop(28)
# df_combined = df_combined.drop(69)
# df_combined = df_combined.drop(74)
# df_combined = df_combined.drop(76)

In [ ]:
df_combined

,source,Network,Date,Time,Show,n,start,end,text_field,count_cc,count_gw,count_all,count_hurricane,landfall_us,diff_date,docid_field
0,df_arthur,MSNBCW,2014-07-01,100000,Morning_Joe.align.srt,345,1301.454,1304.332,YESTERDAY. A HEARTFELT APOLOGY YESTERDAY. FIRS...,0,0,0,0,2014-07-04,-3,1
1,df_arthur,CNNW,2014-07-01,190000,CNN_Newsroom_With_Brooke_Baldwin.align.srt,6,18.685,17.954,AND WE CONTINUE ON OUR HOUR TWO HERE. YOU'RE W...,0,0,0,0,2014-07-04,-3,2
2,df_arthur,CNNW,2014-07-01,200000,The_Lead_With_Jake_Tapper.align.srt,4,6.764,8.607,HERE ON CNN. THANK YOU. THAT'S IT FOR ME. THAN...,0,0,0,0,2014-07-04,-3,3
3,df_arthur,CNNW,2014-07-02,100000,New_Day.align.srt,19,58.790,59.889,NEW YORK AND KATE FOR THE LATEST ON THAT. GOOD...,0,0,0,0,2014-07-04,-2,4
4,df_arthur,FOXNEWSW,2014-07-02,130000,Americas_Newsroom.align.srt,61,303.387,307.638,AND BACK INTO OTHER STATES IN THE MEANTIME. IT...,0,0,0,0,2014-07-04,-2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4365,df_zeta,MSNBCW,2020-10-29,150000,MSNBC_Live_With_Craig_Melvin.align.srt,262,940.655,943.588,"SCIENCE AND THE FACTS. DR. GUPTA, THANK YOU. W...",0,0,0,0,2020-10-28,1,4366
4366,df_zeta,FOXNEWSW,2020-10-29,220000,Special_Report_With_Bret_Baier.align.srt,71,256.141,261.669,WERE GOING ON TO WIN A RECORD SHARE OF THE HIS...,0,0,0,0,2020-10-28,1,4367
4367,df_zeta,CNNW,2020-10-30,95900,New_Day_With_Alisyn_Camerota_and_John_Berman.a...,811,3195.475,3198.016,"APPRECIATE IT, BEN. GOOD TO SEE YOU. YOU TOO. ...",0,0,0,0,2020-10-28,2,4368
4368,df_zeta,MSNBCW,2020-10-30,100000,Morning_Joe.align.srt,2222,8701.047,8704.519,"ALL RIGHT, THE AFGHAN WAR VET IS RUNNING IN A ...",0,0,0,0,2020-10-28,2,4369


Load Embedding Model

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
rows = len(df_combined.index)
values = range(rows)

Main Loop

In [ ]:
for i in values:
  # GET TEXT
  doc = df_combined.iloc[i]["text_field"]
  sentences = doc.split('. ')

  # UNIFY SENTENCE LENGTH
  # Get the length of each sentence
  sentece_length = [len(each) for each in sentences]
  # Determine longest outlier
  long = np.mean(sentece_length) + np.std(sentece_length) *2
  # Determine shortest outlier
  short = np.mean(sentece_length) - np.std(sentece_length) *2
  # Shorten long sentences
  text = ''
  for each in sentences:
      if len(each) > long:
          # let's replace all the commas with dots
          comma_splitted = each.replace(',', '.')
      else:
          text+= f'{each}. '
  sentences = text.split('. ')
  # Now let's concatenate short ones
  text = ''
  for each in sentences:
      if len(each) < short:
          text+= f'{each} '
      else:
          text+= f'{each}. '

  # EMBED SENTENCES
  # Split text into sentences
  sentences = text.split('. ')
  # Embed sentences
  embeddings = model.encode(sentences)
  print(embeddings.shape)

  # CREATE COSINE SIMILARITY MATRIX
  # Create similarities matrix
  similarities = cosine_similarity(embeddings)
  # Lets plot the result we got
  # sns.heatmap(similarities).set_title('Cosine similarities matrix');

  # FIND SPLITTING POINTS
  def rev_sigmoid(x:float)->float:
      return (1 / (1 + math.exp(0.5*x)))

  def activate_similarities(similarities:np.array, p_size=10)->np.array:
          """ Function returns list of weighted sums of activated sentence similarities
          Args:
              similarities (numpy array): it should square matrix where each sentence corresponds to another with cosine similarity
              p_size (int): number of sentences are used to calculate weighted sum
          Returns:
              list: list of weighted sums
          """
          # To create weights for sigmoid function we first have to create space. P_size will determine number of sentences used and the size of weights vector.
          x = np.linspace(-10,10,p_size)
          # Then we need to apply activation function to the created space
          y = np.vectorize(rev_sigmoid)
          # Because we only apply activation to p_size number of sentences we have to add zeros to neglect the effect of every additional sentence and to match the length ofvector we will multiply
          activation_weights = np.pad(y(x),(0,similarities.shape[0]-p_size))
          ### 1. Take each diagonal to the right of the main diagonal
          diagonals = [similarities.diagonal(each) for each in range(0,similarities.shape[0])]
          ### 2. Pad each diagonal by zeros at the end. Because each diagonal is different length we should pad it with zeros at the end
          diagonals = [np.pad(each, (0,similarities.shape[0]-len(each))) for each in diagonals]
          ### 3. Stack those diagonals into new matrix
          diagonals = np.stack(diagonals)
          ### 4. Apply activation weights to each row. Multiply similarities with our activation.
          diagonals = diagonals * activation_weights.reshape(-1,1)
          ### 5. Calculate the weighted sum of activated similarities
          activated_similarities = np.sum(diagonals, axis=0)
          return activated_similarities

  # Let's apply our function. For long sentences i reccomend to use 10 or more sentences
  activated_similarities = activate_similarities(similarities, p_size=10)

  # Let's create empty fig for our plor
  # fig, ax = plt.subplots()
  ### 6. Find relative minima of our vector. For all local minimas and save them to variable with argrelextrema function
  minmimas = argrelextrema(activated_similarities, np.less, order=2) #order parameter controls how frequent should be splits. I would not reccomend changing this parameter.
  # plot the flow of our text with activated similarities
  # sns.lineplot(y=activated_similarities, x=range(len(activated_similarities)), ax=ax).set_title('Relative minimas');
  # Now lets plot vertical lines in order to see where we created the split
  # plt.vlines(x=minmimas, ymin=min(activated_similarities), ymax=max(activated_similarities), colors='purple', ls='--', lw=1, label='vline_multiple - full height')

  # ZOOM IN TO SEE RESULTS
  # fig, ax = plt.subplots()
  # for all local minimals
  # sns.lineplot(y=activated_similarities, x=range(len(activated_similarities)), ax=ax).set_title('Zoomed in Local minimas');
  # Now lets plot vertical line in order to understand if we have done what we wanted
  # plt.vlines(x=minmimas, ymin=min(activated_similarities), ymax=max(activated_similarities), colors='purple', ls='--', lw=1, label='vline_multiple - full height')
  # ax.set_xlim([0,100])

  # SPLIT INTO PARAGRAPHS
  # Create empty string
  split_points = [each for each in minmimas[0]]
  text = ''
  for num,each in enumerate(sentences):
      if num in split_points:
          text+=f'\n\n{each}. '
      else:
          text+=f'{each}. '

  from io import StringIO  # Python3
  import sys
  # Store the reference, in case you want to show things again in standard output
  old_stdout = sys.stdout
  # This variable will store everything that is sent to the standard output
  result = StringIO()
  sys.stdout = result

  print(text)

  # Redirect again the std output to screen
  sys.stdout = old_stdout
  # Then, get the stdout like a string and process it!
  result_string = result.getvalue()

  # MAKE DATAFRAME
  from io import StringIO
  import pandas as pd

  # Convert String into StringIO
  csvStringIO = StringIO(result_string)

  df = pd.read_csv(csvStringIO,
                    sep = '\r\n',
                    header = None,
                    engine = 'python')

  df

  df = df.rename(columns={0: "text_field"})
  df['source'] = df_combined.iloc[i]['source']
  df['Network'] = df_combined.iloc[i]['Network']
  df['Date'] = df_combined.iloc[i]['Date']
  df['Time'] = df_combined.iloc[i]['Time']
  df['Show'] = df_combined.iloc[i]['Show']
  df['n'] = df_combined.iloc[i]['n']
  df['start'] = df_combined.iloc[i]['start']
  df['end'] = df_combined.iloc[i]['end']
  df['count_cc'] = df_combined.iloc[i]['count_cc']
  df['count_gw'] = df_combined.iloc[i]['count_gw']
  df['count_all'] = df_combined.iloc[i]['count_all']
  df['count_hurricane'] = df_combined.iloc[i]['count_hurricane']
  df['landfall_us'] = df_combined.iloc[i]['landfall_us']
  df['diff_date'] = df_combined.iloc[i]['diff_date']
  df['docid_field'] = df_combined.iloc[i]['docid_field']
  df_all = df_all.append(df, ignore_index=True)

(25, 768)
(27, 768)


<ipython-input-12-d820f0badc5a>:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df, ignore_index=True)
<ipython-input-12-d820f0badc5a>:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df, ignore_index=True)


(47, 768)


<ipython-input-12-d820f0badc5a>:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df, ignore_index=True)


(144, 768)
(61, 768)


...

(48, 768)
(26, 768)
(36, 768)
(68, 768)


<ipython-input-12-d820f0badc5a>:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df, ignore_index=True)
<ipython-input-12-d820f0badc5a>:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df, ignore_index=True)


(43, 768)
(7, 768)


<ipython-input-12-d820f0badc5a>:148: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df, ignore_index=True)


ValueError: ignored

In [ ]:
df_all

,text_field,source,Network,Date,Time,Show,n,start,end,count_cc,count_gw,count_all,count_hurricane,landfall_us,diff_date,docid_field
0,YESTERDAY.,df_arthur,MSNBCW,2014-07-01,100000,Morning_Joe.align.srt,345,1301.454,1304.332,0,0,0,0,2014-07-04,-3,1
1,"A HEARTFELT APOLOGY YESTERDAY. FIRST, BILL KAR...",df_arthur,MSNBCW,2014-07-01,100000,Morning_Joe.align.srt,345,1301.454,1304.332,0,0,0,0,2014-07-04,-3,1
2,THE STORM COULD BE AND IS FORECASTED BY THE HU...,df_arthur,MSNBCW,2014-07-01,100000,Morning_Joe.align.srt,345,1301.454,1304.332,0,0,0,0,2014-07-04,-3,1
3,"THAT'S THE OFFICIAL FORECAST, SOMEWHERE VERY C...",df_arthur,MSNBCW,2014-07-01,100000,Morning_Joe.align.srt,345,1301.454,1304.332,0,0,0,0,2014-07-04,-3,1
4,THIS WIND STORM PROBABLY DID MORE DAMAGE THAN ...,df_arthur,MSNBCW,2014-07-01,100000,Morning_Joe.align.srt,345,1301.454,1304.332,0,0,0,0,2014-07-04,-3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47737,IT'S CLOSE ENOUGH. THIS WILL BE 100 MILES AN H...,df_arthur,CNNW,2014-07-03,170000,Wolf.align.srt,1,1.340,3.302,0,0,0,0,2014-07-04,-1,28
47738,THEY'RE TELLING RESIDENTS AND TOURISTS TO PREP...,df_arthur,CNNW,2014-07-03,170000,Wolf.align.srt,1,1.340,3.302,0,0,0,0,2014-07-04,-1,28
47739,"THANKS FOR THE WARNING, CHAD. STAY WITH CNN AL...",df_arthur,CNNW,2014-07-03,170000,Wolf.align.srt,1,1.340,3.302,0,0,0,0,2014-07-04,-1,28
47740,A JUDGE WILL HEAR TESTIMONY ON WHETHER TO GRAN...,df_arthur,CNNW,2014-07-03,170000,Wolf.align.srt,1,1.340,3.302,0,0,0,0,2014-07-04,-1,28


In [ ]:
from google.colab import files
df_all.to_csv('df_all.csv', encoding = 'utf-8-sig')
files.download('df_all.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>